In [2]:
!pip install -q -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.3 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

# Przykładowe dane
import numpy as np
X = np.random.rand(1000, 10)
y = np.random.rand(1000, 1)

# Funkcja budująca model
def build_model(hp):
    model = keras.Sequential()

    # Liczba warstw: od 1 do 3
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(
            keras.layers.Dense(
                # Liczba neuronów: od 16 do 128, skok o 16
                units=hp.Int('units_' + str(i), min_value=16, max_value=128, step=16),
                # Funkcja aktywacji: do wyboru z 'relu', 'tanh', 'sigmoid'
                activation=hp.Choice('activation_' + str(i), ['relu', 'tanh', 'sigmoid'])
            )
        )

    model.add(keras.layers.Dense(1))  # Warstwa wyjściowa dla regresji

    # Kompilowanie modelu
    model.compile(
        optimizer=keras.optimizers.Adam(
            # Współczynnik uczenia: logarytmiczna przestrzeń wartości od 1e-4 do 1e-2
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        ),
        loss='mse',  # Średni błąd kwadratowy
        metrics=['mae']  # Średni błąd bezwzględny
    )

    return model

# Tworzenie tunera
tuner = kt.RandomSearch(
    build_model,
    objective='val_mae',
    max_trials=10,  # Liczba prób
    executions_per_trial=2,  # Liczba wykonanych prób dla każdego zestawu hiperparametrów
    directory='my_dir',
    project_name='regression_tuning'
)

# Podział danych na treningowe i walidacyjne
X_train, X_val = X[:800], X[800:]
y_train, y_val = y[:800], y[800:]

# Przeprowadzanie tuningu
tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val))

# Najlepszy model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Najlepsza liczba warstw: {best_hps.get('num_layers')}")
print(f"Liczba neuronów: {[best_hps.get(f'units_{i}') for i in range(best_hps.get('num_layers'))]}")
print(f"Funkcje aktywacji: {[best_hps.get(f'activation_{i}') for i in range(best_hps.get('num_layers'))]}")
print(f"Learning rate: {best_hps.get('learning_rate')}")



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
1                 |1                 |num_layers
32                |32                |units_0
relu              |relu              |activation_0
0.00036762        |0.00036762        |learning_rate

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.1793 - mae: 0.3501 - val_loss: 0.1199 - val_mae: 0.2862
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1259 - mae: 0.3016 - val_loss: 0.1008 - val_mae: 0.2692
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0964 - mae: 0.2669 - val_loss: 0.0986 - val_mae: 0.2678
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1046 - mae: 0.2754 - val_loss: 0.0970 - val_mae: 0.2662
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0987 - mae: 0.2677 - val_loss: 0.0956 - val_mae: 0.2647
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0933 - mae: 0.2610 - val_loss: 0.0945 - val_mae: 0.2636
Epoch 7/50
25/25 ━━━━━━━

KeyboardInterrupt: 